statistics-resampling online 
============

Interact with Octave and the statistics-resampling package online in Jupyter Notebook with Binder. All commands are interpreted by Octave.  Since this is a [MetaKernel](https://github.com/Calysto/metakernel), a standard set of magics are available.  Help on commands is available using the `%help` magic or using `?` with a command.

In [ ]:
#################################### YOU MUST RUN THIS CELL FIRST ####################################

info = ver;
try
  addpath('package-functions');
catch
  pkg install "https://github.com/gnu-octave/statistics-resampling/archive/refs/heads/master.zip"
  pkg load statistics-resampling
end
fprintf('Running version statistics-resampling (version 5.5.7) in GNU Octave %s...\n', info.Version);

######################################################################################################

In [ ]:
####################################### ENTER YOUR CODE BELOW ########################################




